# 构建图像生成应用

大语言模型（LLM）不仅仅能生成文本，还可以根据文本描述生成图像。图像作为一种模态，在医疗科技、建筑、旅游、游戏开发等众多领域都非常有用。本章将介绍目前最流行的两种图像生成模型：DALL-E 和 Midjourney。

## 简介

本节内容包括：

- 图像生成及其应用价值
- DALL-E 和 Midjourney 的原理与用法
- 如何构建一个图像生成应用

## 学习目标

完成本节后，你将能够：

- 构建一个图像生成应用
- 通过元提示（meta prompts）为你的应用设定边界
- 使用 DALL-E 和 Midjourney 进行图像生成

## 为什么要构建图像生成应用？

图像生成应用是探索生成式 AI 能力的绝佳方式。例如，它们可以用于：

- **图像编辑与合成**。你可以为各种场景生成图像，比如图片编辑、图片合成等。

- **适用于多个行业**。还可以为医疗科技、旅游、游戏开发等多个行业生成所需的图片。

## 场景：Edu4All

在本节中，我们将继续以我们的创业公司 Edu4All 为例。学生们可以为他们的作业生成图片，具体生成什么图片由学生自己决定，比如为自己的童话故事画插图、为故事创造新角色，或者帮助他们把自己的想法和概念可视化。

比如，Edu4All 的学生在课堂上学习纪念碑时，可以生成如下图片：

![Edu4All startup, class on monuments, Eifel Tower](../../../../translated_images/startup.94d6b79cc4bb3f5afbf6e2ddfcf309aa5d1e256b5f30cc41d252024eaa9cc5dc.zh.png)

使用类似这样的提示词：

> “清晨阳光下，狗站在埃菲尔铁塔旁”

## 什么是 DALL-E 和 Midjourney？

[DALL-E](https://openai.com/dall-e-2?WT.mc_id=academic-105485-koreyst) 和 [Midjourney](https://www.midjourney.com/?WT.mc_id=academic-105485-koreyst) 是目前最流行的两种图像生成模型，你可以通过输入提示词来生成图片。

### DALL-E

我们先来看 DALL-E，这是一种可以根据文本描述生成图片的生成式 AI 模型。

> [DALL-E 结合了两种模型：CLIP 和扩散注意力](https://towardsdatascience.com/openais-dall-e-and-clip-101-a-brief-introduction-3a4367280d4e?WT.mc_id=academic-105485-koreyst)。

- **CLIP** 是一种可以从图片和文本中生成嵌入（即数据的数值表示）的模型。

- **扩散注意力** 是一种可以根据嵌入生成图片的模型。DALL-E 在大量图片和文本的数据集上进行训练，可以根据文本描述生成图片。例如，DALL-E 可以生成戴帽子的猫，或者有莫霍克发型的狗的图片。

### Midjourney

Midjourney 的工作方式和 DALL-E 类似，也是根据文本提示生成图片。你同样可以用类似“戴帽子的猫”或“有莫霍克发型的狗”这样的提示词来生成图片。

![Image generated by Midjourney, mechanical pigeon](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8c/Rupert_Breheny_mechanical_dove_eca144e7-476d-4976-821d-a49c408e4f36.png/440px-Rupert_Breheny_mechanical_dove_eca144e7-476d-4976-821d-a49c408e4f36.png?WT.mc_id=academic-105485-koreyst)

*图片来源：Wikipedia，图片由 Midjourney 生成*

## DALL-E 和 Midjourney 的工作原理

首先来看 [DALL-E](https://arxiv.org/pdf/2102.12092.pdf?WT.mc_id=academic-105485-koreyst)。DALL-E 是一种基于 Transformer 架构的生成式 AI 模型，采用了*自回归 Transformer*。

*自回归 Transformer* 决定了模型如何根据文本描述生成图片：它一次生成一个像素，然后用已生成的像素继续生成下一个像素。这个过程会经过神经网络的多层，直到图片生成完成。

通过这种方式，DALL-E 可以控制生成图片中的属性、物体、特征等。不过，DALL-E 2 和 3 在图片生成的可控性上更强，


## 构建你的第一个图像生成应用

那么，如何构建一个图像生成应用呢？你需要以下几个库：

- **python-dotenv**，强烈建议你使用这个库，把你的密钥保存在 *.env* 文件中，避免直接写在代码里。
- **openai**，你将用这个库与 OpenAI API 进行交互。
- **pillow**，用于在 Python 中处理图片。
- **requests**，帮助你发送 HTTP 请求。

1. 新建一个 *.env* 文件，内容如下：

    ```text
    AZURE_OPENAI_ENDPOINT=<your endpoint>
    AZURE_OPENAI_API_KEY=<your key>
    ```

    你可以在 Azure Portal 的资源“密钥和终端”部分找到这些信息。


1. 将上述库收集到一个名为 *requirements.txt* 的文件中，如下所示：

    ```text
    python-dotenv
    openai
    pillow
    requests
    ```

1. 接下来，创建虚拟环境并安装这些库：


In [ ]:
# create virtual env
! python3 -m venv venv
# activate environment
! source venv/bin/activate
# install libraries
# pip install -r requirements.txt, if using a requirements.txt file 
! pip install python-dotenv openai pillow requests

> [!NOTE]
> 在 Windows 上，请使用以下命令来创建并激活你的虚拟环境：

    ```bash
    python3 -m venv venv
    venv\Scripts\activate.bat
    ```

1. 在名为 *app.py* 的文件中添加以下代码：

    ```python
    import openai
    import os
    import requests
    from PIL import Image
    import dotenv
    
    # import dotenv
    dotenv.load_dotenv()
    
    # Get endpoint and key from environment variables
    openai.api_base = os.environ['AZURE_OPENAI_ENDPOINT']
    openai.api_key = os.environ['AZURE_OPENAI_API_KEY']     
    
    # Assign the API version (DALL-E is currently supported for the 2023-06-01-preview API version only)
    openai.api_version = '2023-06-01-preview'
    openai.api_type = 'azure'
    
    
    try:
        # Create an image by using the image generation API
        generation_response = openai.Image.create(
            prompt='Bunny on horse, holding a lollipop, on a foggy meadow where it grows daffodils',    # Enter your prompt text here
            size='1024x1024',
            n=2,
            temperature=0,
        )
        # Set the directory for the stored image
        image_dir = os.path.join(os.curdir, 'images')
    
        # If the directory doesn't exist, create it
        if not os.path.isdir(image_dir):
            os.mkdir(image_dir)
    
        # Initialize the image path (note the filetype should be png)
        image_path = os.path.join(image_dir, 'generated-image.png')
    
        # Retrieve the generated image
        image_url = generation_response["data"][0]["url"]  # extract image URL from response
        generated_image = requests.get(image_url).content  # download the image
        with open(image_path, "wb") as image_file:
            image_file.write(generated_image)
    
        # Display the image in the default image viewer
        image = Image.open(image_path)
        image.show()
    
    # catch exceptions
    except openai.InvalidRequestError as err:
        print(err)

    ```

下面来解释一下这段代码：

- 首先，我们导入了所需的库，包括 OpenAI 库、dotenv 库、requests 库和 Pillow 库。

    ```python
    import openai
    import os
    import requests
    from PIL import Image
    import dotenv
    ```

- 接下来，我们从 *.env* 文件中加载环境变量。

    ```python
    # import dotenv
    dotenv.load_dotenv()
    ```

- 然后，我们设置 OpenAI API 的 endpoint、key、版本和类型。

    ```python
    # Get endpoint and key from environment variables
    openai.api_base = os.environ['AZURE_OPENAI_ENDPOINT']
    openai.api_key = os.environ['AZURE_OPENAI_API_KEY'] 

    # add version and type, Azure specific
    openai.api_version = '2023-06-01-preview'
    openai.api_type = 'azure'
    ```

- 接下来，我们生成图片：

    ```python
    # Create an image by using the image generation API
    generation_response = openai.Image.create(
        prompt='Bunny on horse, holding a lollipop, on a foggy meadow where it grows daffodils',    # Enter your prompt text here
        size='1024x1024',
        n=2,
        temperature=0,
    )
    ```

    上面的代码会返回一个包含生成图片 URL 的 JSON 对象。我们可以用这个 URL 下载图片并保存到文件中。

- 最后，我们打开图片，并用系统默认的图片查看器显示出来：

    ```python
    image = Image.open(image_path)
    image.show()
    ```
    
### 生成图片的更多细节

让我们更详细地看看生成图片的代码：

```python
generation_response = openai.Image.create(
        prompt='Bunny on horse, holding a lollipop, on a foggy meadow where it grows daffodils',    # Enter your prompt text here
        size='1024x1024',
        n=2,
        temperature=0,
    )
```

- **prompt**，是用于生成图片的文本提示。在这个例子中，我们用的提示是 “Bunny on horse, holding a lollipop, on a foggy meadow where it grows daffodils”（骑在马上的兔子，手里拿着棒棒糖，在长着水仙花的雾气草地上）。
- **size**，是生成图片的尺寸。本例中我们生成的是 1024x1024 像素的图片。
- **n**，是生成图片的数量。本例中我们生成了两张图片。
- **temperature**，是控制生成式 AI 模型输出随机性的参数。temperature 的取值范围是 0 到 1，0 表示输出是确定性的，1 表示输出是完全随机的。默认值是 0.7。

关于图片你还能做更多的事情，我们会在下一节介绍。

## 图片生成的更多功能

到目前为止，你已经看到我们只用几行 Python 代码就能生成图片。不过，其实你还能对图片做更多操作。

你还可以：

- **进行编辑**。通过提供一张已有图片、一个遮罩和一个提示词，你可以修改图片。例如，你可以在图片的某个部分加点东西。比如我们的兔子图片，你可以给兔子加顶帽子。实现方法是提供图片、一个遮罩（标记需要更改的区域）和一个文本提示，说明要做什么。

    ```python
    response = openai.Image.create_edit(
      image=open("base_image.png", "rb"),
      mask=open("mask.png", "rb"),
      prompt="An image of a rabbit with a hat on its head.",
      n=1,
      size="1024x1024"
    )
    image_url = response['data'][0]['url']
    ```

    原始图片只有兔子，最终图片则会在兔子头上加上帽子。
    
- **创建变体**。  
    查看我们的 [OpenAI 笔记本获取更多信息](./oai-assignment.ipynb?WT.mc_id=academic-105485-koreyst)。



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
